In [2]:
#This version of the evolutionary algorithm seeks to implement
#step 7, i.e. the production efficiency curve
#and result smoothing, such that stray zeroes and ones are replaced by their neighbours

In [3]:
#Production efficiency techs take 150 days base, construction and industry take 200 base

#Meta strat:
#21 march 1936: Improved worker conditions
#2 september 1936: War eco
#27 november 1936: Popular figurehead
#30 march 1937: Captain of industry, state 12


#Normal strat:
#17 july 1936: Spanish civil war, war eco


#Research:
#23 may 1936: Basic machine tools, state 4
#8 july 1936: Construction 1, state 5
#24 november 1936: Concentrated 1, state 9 
#28 feburari 1937: Construction 2, state 12
#11 april 1937: Improved machine tools, state 13
#25 may 1937: Concentrated 2, state 14

#ASSUMPTION FOR NOW: All further tech is unlocked 3 months after they become current TEST IN GAME
#Concentrated 3, Machine tools 3, Construction 3, state 33
#Concentrated 4, Machine tools 4, Construction 4, state 54

In [4]:
#STABILITY EVENTS:
#State 2: +5%
#State 4: +5%
#State 8: +10%

In [5]:
import numpy as np

In [6]:
def selection(pop, scores, k=3):
    selection_ix = np.random.randint(len(pop))
    for ix in np.random.randint(0, len(pop), k-1):
        if scores[ix] > scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

def crossover(p1, p2, r_cross):
    c1, c2 = p1.copy(), p2.copy()
    if np.random.rand() < r_cross:
        pt = np.random.randint(1, len(p1)-1)
        #print(pt)
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
        #visualise what solutions have been tried
    return [c1, c2]

def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        if np.random.rand() < r_mut:
            bitstring[i] = 1 - bitstring[i]
    return bitstring

In [106]:
def update_consumergoods(state, toaster_percent):
    match state:
        case 5:
            toaster_percent -= 0.15 #WAR ECO        
    return round(toaster_percent, 2)

def update_constructionspeed(state, civ_output_modifier):
    match state:
        case 0:
            civ_output_modifier += 0.15
        case 5:
            civ_output_modifier += 0.1
        case 12:
            civ_output_modifier += 0.2 #tech + captain of industry
        case 33:
            civ_output_modifier += 0.1
        case 54:
            civ_output_modifier += 0.1
    return round(civ_output_modifier, 2)

def update_miloutput(state, mil_output_modifier):
    match state:
        case 9:
            mil_output_modifier += 0.15
        case 14:
            mil_output_modifier += 0.15
        case 33:
            mil_output_modifier += 0.15
        case 54:
            mil_output_modifier += 0.15
    return round(mil_output_modifier, 2)

def update_prodefficiency(state, production_efficiency_cap):
    match state:
        case 4:
            production_efficiency_cap += 0.1
        case 13:
            production_efficiency_cap += 0.1
        case 33:
            production_efficiency_cap += 0.1
        case 54:
            production_efficiency_cap += 0.1
    return round(production_efficiency_cap, 2)

def update_civcost(state, civ_construction_modifier):
    match state:
        case 5:
            civ_construction_modifier += 0.3
    return round(civ_construction_modifier, 2)

def update_milcost(state, mil_construction_modifier):
    match state:
        case 5:
            mil_construction_modifier += 0.5
    return round(mil_construction_modifier, 2)

In [107]:
def production(actions):
    mils = 32
    civs = 45
    total = mils + civs
    consumergoods_percentage = 0.40
    consumergoods = np.floor(total * consumergoods_percentage)
    available_civs = np.maximum(int(civs - consumergoods), 0)
    
    #Civ output modifiers are additive (-30% + 15% = -15%), multiplied by state infrastructure
    civ_output = 5 
    civ_output_modifier = 1 #Construction speed
    civ_construction_modifier = 0.7
    mil_construction_modifier = 0.7
    mil_output = 4.5
    mil_output_modifier = 1
    #ic_produced = mil_output * (mil_output_modifier) * production efficiency
    prod_eff = 0.5
    production_efficiency_cap = 0.55
    
    #Investigate how new factories add to production efficiency
    #production_efficiency works as on the wiki, except for the starting factories
    #with different rules
    #The number of factories because of trade is quite significant, but ultimately unpredictable
    
    state = 0
    state_size_days = 35 #100 #how many days per state
    ic_produced = 0
    ic_total = 0    
    civ_purchase_pool = 0 
    mil_purchase_pool = 0
    civ_to_mil_pool = 0
    mil_to_civ_pool = 0
    infrastructure_modifier = 1.6
    
    for i in range(len(actions)):
        consumergoods_percentage = (update_consumergoods(i, consumergoods_percentage))
        civ_output_modifier = (update_constructionspeed(i, civ_output_modifier))
        mil_output_modifier = (update_miloutput(i, mil_output_modifier))
        civ_construction_modifier = (update_civcost(i, civ_construction_modifier))
        mil_construction_modifier = (update_milcost(i, mil_construction_modifier))
        production_efficiency_cap = (update_prodefficiency(i, production_efficiency_cap))
        
        ic_produced = mil_output * mils * state_size_days * mil_output_modifier * prod_eff
        
        if actions[i] == 0: #build mils     
            mil_purchase_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * mil_construction_modifier)
            
            existing_mils = mils #Used exclusively for production efficiency calculations
            mils += np.floor_divide(mil_purchase_pool, 7200) 
            total += np.floor_divide(mil_purchase_pool, 7200) 
            consumergoods = np.ceil(total * consumergoods_percentage)
            mil_purchase_pool = np.mod(mil_purchase_pool, 7200)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            
            prod_eff = (existing_mils * prod_eff + (mils - existing_mils) * 0.1) / mils
            growth = 0.001 * (production_efficiency_cap **2 / prod_eff)
            prod_eff += (growth * 35) / 2
            prod_eff = min(1, prod_eff)
            
            state += 1
            ic_total += ic_produced
        if actions[i] == 1: #build civs         
            civ_purchase_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * civ_construction_modifier)
            civs += np.floor_divide(civ_purchase_pool, 7200) 
            total += np.floor_divide(civ_purchase_pool, 7200) 
            consumergoods = np.ceil(total * consumergoods_percentage)
            civ_purchase_pool = np.mod(civ_purchase_pool, 7200)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            
            growth = 0.001 * (production_efficiency_cap **2 / prod_eff)
            prod_eff += (growth * 35) / 2
            prod_eff = min(1, prod_eff)
            
            state += 1
            ic_total += ic_produced
        #print(available_civs)
        #print(civs, mils, consumergoods)

        if actions[i] == 2: #Convert civs to mils
            ic_total += ic_produced
        
        if actions[i] == 3: #Convert mils to civs
            ic_total += ic_produced
        
    return (ic_total, mils, civs)

In [108]:
production([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

(60751.73169614916, 50.0, 45)

In [109]:
test = [0] * 57
test2 = [1] * 16 + [0] * 41
test3 = [1] * 57

production(test), production(test2), production(test3)

((530856.7617686796, 93.0, 45),
 (516455.10286099, 114.0, 64.0),
 (316115.34711203014, 32, 324.0))

In [110]:
def genetic_algorithm(n_bits, n_pop, n_iter, r_cross):
    pop = [np.random.randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
    best, best_eval = 0, production(pop[0])
    
    for gen in range(n_iter):
        scores = [production(c) for c in pop]
        #print(pop, scores)
        for i in range(n_pop):
            if scores[i] > best_eval:
                best, best_eval = pop[i], scores[i]
                print("Generation " + str(gen) + ": new best solution found!")
                print(best, best_eval)
                
        #ADD ELITISM PLUS ADDITIONAL COMPLETELY RANDOM SAMPLES HERE
        
        selected = [selection(pop, scores) for _ in range(n_pop)]
        children = []
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i+1]
            c = crossover(p1, p2, r_cross)
            for child in c:
                mutated = mutation(child, 1 / n_bits)
                children.append(mutated)
        #print(np.mean(sum([production(c) for c in pop])))
        #print(np.mean(sum([production(c) for c in children])))
        #print(sum([sum(strat) for strat in pop]))
        pop = children
        
    #SMOOTHING TEST
    best_smoothed = [best[0]]
    for i in range(1, len(best)-1):
        #print((best[i-1],best[i],best[i+1]))
        if best[i-1] == best[i+1]:
            best_smoothed.append(best[i-1])
        else:
            best_smoothed.append(best[i])
    best_smoothed.append(best[-1])
    best_smoothed_eval = production(best_smoothed)
    if best_smoothed_eval > best_eval:
        print("Smoothing : new best solution found!")
        print(best_smoothed, best_smoothed_eval)
        return [best_smoothed, best_smoothed_eval]
    
    return [best, best_eval]

In [111]:
genetic_algorithm(10,10, 5, 0.9) #There should be 66 states of 30 days, ends up on June 5th '41
#Note: states could also be 35 days to reflect the length of the focus tree
#This would be 57 states of 35 days, ending onJune 19th '41 (no leap years)

Generation 0: new best solution found!
[0, 1, 1, 0, 0, 0, 0, 1, 0, 0] (28581.63461502384, 38.0, 47.0)
Generation 0: new best solution found!
[0, 1, 0, 0, 1, 0, 0, 1, 0, 0] (28613.927937013956, 38.0, 47.0)
Generation 1: new best solution found!
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0] (28684.939653292033, 40.0, 45.0)
Generation 2: new best solution found!
[0, 0, 0, 0, 1, 0, 0, 0, 1, 1] (28697.406091532877, 37.0, 47.0)
Generation 3: new best solution found!
[0, 0, 0, 0, 1, 0, 0, 0, 0, 1] (28720.100605200147, 38.0, 46.0)
Generation 3: new best solution found!
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1] (28806.306006584557, 39.0, 46.0)
Generation 4: new best solution found!
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (28806.306006584557, 40.0, 45)


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], (28806.306006584557, 40.0, 45)]

In [112]:
genetic_algorithm(57,100, 30, 0.9)

Generation 0: new best solution found!
[1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1] (480689.6098421975, 96.0, 102.0)
Generation 0: new best solution found!
[0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0] (487042.71405027097, 99.0, 90.0)
Generation 0: new best solution found!
[1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0] (487585.7216227782, 101.0, 88.0)
Generation 0: new best solution found!
[0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0] (502474.0565117224, 97.0, 72.0)
Generation 0: new best so

Generation 21: new best solution found!
[1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0] (531308.8304397255, 93.0, 49.0)
Generation 21: new best solution found!
[1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0] (531385.3507455962, 94.0, 48.0)
Generation 24: new best solution found!
[1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0] (531431.5785435905, 94.0, 48.0)
Generation 24: new best solution found!
[1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0] (531605.1566154769, 95.0, 47.0)
Generation 29: new best 

[[1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 (531682.1055111538, 95.0, 47.0)]

In [113]:
#Building civs for the first 16 states (roughly 1.5 years) seems to be it's optimal solution
#Building civs for the first 12 states (roughly 1.25 years) seems to also be an optimal solution
#Interestingly, building civs for 13, 14 or 15 performs worse than either, ergo the solution space has local maxima
#How do you account for the fact that the second solution has better long-term potential?
#How can I scale the factory scores as an added weight?
#Check in game if the difference of four states really equals 7 civilian factories

max_12 = production([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
max_16 = production([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
max_12, max_16

((525060.049651363, 106.0, 56.0), (516455.10286099, 114.0, 64.0))

In [114]:
production([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1])

(523928.7586402615, 95.0, 57.0)

In [29]:
base_efficiency = 0.5
efficiency_cap = 0.6
new_efficiency = 0.1
mils = 10
new_mils = 1
total_mils = mils + new_mils
efficiency = round((base_efficiency * mils + new_mils * new_efficiency) / total_mils, 4)
efficiency_start = efficiency

for i in range(35):
    growth = 0.001 * (efficiency_cap **2 / efficiency)
    efficiency += growth
    print(round(efficiency, 4), round(growth, 7))
    
print(round(efficiency - efficiency_start, 4))

0.4644 0.0007765
0.4652 0.0007752
0.4659 0.0007739
0.4667 0.0007727
0.4675 0.0007714
0.4682 0.0007701
0.469 0.0007688
0.4698 0.0007676
0.4705 0.0007663
0.4713 0.0007651
0.4721 0.0007638
0.4728 0.0007626
0.4736 0.0007614
0.4744 0.0007601
0.4751 0.0007589
0.4759 0.0007577
0.4766 0.0007565
0.4774 0.0007553
0.4781 0.0007541
0.4789 0.0007529
0.4796 0.0007517
0.4804 0.0007506
0.4811 0.0007494
0.4819 0.0007482
0.4826 0.0007471
0.4834 0.0007459
0.4841 0.0007447
0.4849 0.0007436
0.4856 0.0007425
0.4864 0.0007413
0.4871 0.0007402
0.4878 0.0007391
0.4886 0.000738
0.4893 0.0007368
0.49 0.0007357
0.0264


In [30]:
growth = 0.001 * (efficiency_cap **2 / efficiency)
efficiency += (growth * 35) / 2
    
print(round(efficiency - efficiency_start, 4))

0.0522
